In [11]:
import numpy as np
import matplotlib.pyplot as plt
from arlbench.core.algorithms import DQN, PPO, SAC
from pathlib import Path
import os
import pandas as pd
import gpflow
from sklearn.base import BaseEstimator
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import trim_mean
from ConfigSpace import ConfigurationSpace, Float, Categorical
from arlbench.core.algorithms import DQN, PPO, SAC
from autorl_landscape.visualize import (
    FIGSIZES,
    LEGEND_FSIZE,
    TITLE_FSIZE,
    visualize_nd,
)
from matplotlib.gridspec import GridSpecFromSubplotSpec
from itertools import zip_longest
from pandas import DataFrame
from autorl_landscape.analyze.visualization import Visualization

In [12]:
replacement_dict = {"atari_qbert": "atari", 'atari_double_dunk': "atari", 'atari_phoenix': "atari", 'atari_this_game': "atari", 
                    'box2d_lunar_lander': "box2d", "box2d_bipedal_walker": "box2d", 'cc_acrobot': "cc", 'cc_cartpole': "cc", 'cc_mountain_car': "cc", "cc_pendulum": "cc", 'cc_continuous_mountain_car': "cc",
                    'minigrid_door_key': "minigrid", 'minigrid_empty_random': "minigrid", 'minigrid_four_rooms': "minigrid", 
                    'minigrid_unlock': "minigrid", "brax_ant": "brax", "brax_halfcheetah": "brax", "brax_hopper": "brax", "brax_humanoid": "brax"}
algorithms = {"dqn": DQN, "ppo": PPO, "sac": SAC}
envs = ["atari_qbert", 'atari_double_dunk', 'atari_phoenix', 'atari_this_game', 
        'box2d_lunar_lander', 'box2d_bipedal_walker', 'cc_acrobot', 'cc_cartpole', 'cc_mountain_car', 'cc_pendulum', 'cc_continuous_mountain_car',
        'minigrid_door_key', 'minigrid_empty_random', 'minigrid_four_rooms', 'minigrid_unlock', 'brax_ant', 'brax_halfcheetah', 'brax_hopper', 'brax_humanoid']


In [13]:
def _transpose(ll):
    """Transposes lists of lists (or other things you can iterate over)."""
    return list(map(list, zip_longest(*ll, fillvalue=None)))

In [14]:
def iqm(x, axis: int | None = None):
    """Calculate the interquartile mean (IQM) of x. Return nan where np.mean would also write nan."""
    iqms = trim_mean(x, proportiontocut=0.25, axis=axis)
    means = np.mean(x, axis=axis)
    iqms[np.isnan(means)] = np.nan
    return iqms

def estimate_model_fit(X, y, k: int = 5, metrics: list[callable] | None = None) -> pd.DataFrame:            
    if metrics is None:
        metrics = [mean_squared_error, mean_absolute_error]

    cv = KFold(n_splits=k, shuffle=True, random_state=0)

    data = []
    for i, (train_index, test_index) in enumerate(cv.split(X=X, y=y)):
        X_i = X[train_index]
        Y_i = y[train_index]
        model = gpflow.models.GPR((X_i, Y_i), kernel=gpflow.kernels.SquaredExponential())
        opt = gpflow.optimizers.Scipy()
        opt.minimize(model.training_loss, model.trainable_variables)
        f_mean, _ = model.predict_f(X[test_index])
        y_pred = f_mean.numpy()
        results = {}
        results["fold"] = i
        for metric in metrics:
            results[metric.__name__] = metric(y[test_index], y_pred)
        data.append(results)
    data = pd.DataFrame(data)

    return data

class TripleGPModel(BaseEstimator):
    """Triple GP Model."""

    def __init__(
        self,
        data,
        dtype: type,
        y_col: str = "performance",
        y_bounds: tuple[float, float] | None = None,
        best_conf = None,
        hp_names = None,
        configspace: ConfigurationSpace = None,
        ci: float = 0.95,
    ) -> None:
        super().__init__()#data, dtype, y_col, y_bounds, best_conf, ci)
        gpflow.config.set_default_float(dtype)  # WARNING Global!
        self.data = data
        self.y_col = y_col
        self.y_bounds = y_bounds
        self.ci = ci
        self.best_conf = best_conf
        self.model_layer_names = ["upper", "middle", "lower"]
        self.hp_names = hp_names
        self.dtype = dtype
        self.y_info = "yelp"

        # group runs with the same configuration:
        self.dim_info = hp_names
        conf_groups = data.groupby(["run_id"] + hp_names)
        # all groups (configurations):
        self.x = np.array(list(conf_groups.groups.keys()))[:, 1:]
        """(num_confs, num_ls_dims). LS dimensions are sorted by name"""
        # all evaluations (y values) for a group (configuration):
        #print(conf_groups[y_col].apply(list).values)
        y = np.concatenate(conf_groups[y_col].apply(list))
        y = np.array([np.array(ys) for ys in y])
        #y = y.reshape((-1, len(np.unique(data["seed"]))))
        # handle crashed runs by assigning 0 return:
        self.crashed = np.isnan(y)
        y[np.isnan(y)] = 0

        # scale ls dims into [0, 1] interval:
        for i in range(len(hp_names)):
            if configspace.get_hyperparameter(hp_names[i].split(".")[-1]).__class__ == Float:
                self.x[:, i] = (self.x[:, i] - configspace.get_hyperparameter(hp_names[i]).lower) / (configspace.get_hyperparameter(hp_names[i]).upper - configspace.get_hyperparameter(hp_names[i]).lower)
        # scale y into [0, 1] interval:
        self.y = (y - y.min()) / (y.max() - y.min())
        """(num_confs, samples_per_conf)"""

        # just all the single evaluation values, not grouped (but still scaled to [0, 1] interval):
        self.x_samples = np.repeat(self.x, self.y.shape[1], axis=0)
        """(num_confs * samples_per_conf, num_ls_dims)"""
        self.y_samples = self.y.reshape(-1, 1)
        """(num_confs * samples_per_conf, 1)"""

        upper_quantile = 1 - ((1 - ci) / 2)
        lower_quantile = 0 + ((1 - ci) / 2)

        # statistical information about each configuration:
        self.y_iqm = iqm(self.y, axis=1).reshape(-1, 1)
        """(num_confs, 1)"""
        # first, select ci quantile:
        self.y_ci_upper = np.quantile(self.y, upper_quantile, method="median_unbiased", axis=1, keepdims=True)
        """(num_confs, 1)"""
        self.y_ci_lower = np.quantile(self.y, lower_quantile, method="median_unbiased", axis=1, keepdims=True)
        """(num_confs, 1)"""

        self._viz_infos: list[Visualization] = [
            Visualization(
                "Raw Return Samples",
                "scatter",
                "graphs",
                self.build_df(self.x_samples, self.y_samples, "performance"),
                {},
                # {"color": "red"},
            )
        ]

    def fit(self):
        """Fit the three GPs to IQM, upper and lower CI."""
        self.iqm_model = gpflow.models.GPR((self.x, self.y_iqm), kernel=gpflow.kernels.SquaredExponential())
        self.upper_model = gpflow.models.GPR((self.x, self.y_ci_upper), kernel=gpflow.kernels.SquaredExponential())
        self.lower_model = gpflow.models.GPR((self.x, self.y_ci_lower), kernel=gpflow.kernels.SquaredExponential())

        opt = gpflow.optimizers.Scipy()
        opt.minimize(self.iqm_model.training_loss, self.iqm_model.trainable_variables)
        opt.minimize(self.upper_model.training_loss, self.upper_model.trainable_variables)
        opt.minimize(self.lower_model.training_loss, self.lower_model.trainable_variables)

    def estimate_iqm_fit(self):
        print("-"*50)
        print("Estimate IQM surface fit")
        data = estimate_model_fit(X=self.x, y=self.y_iqm, k=5)
        for c in data.columns:
            if c is not "fold":
                print(c, data[c].mean(), data[c].std())
        return data
        
    def get_upper(self, x, assimilate_factor: float = 1.0):
        """Return the upper CI estimate of y at the position(s) x."""
        f_mean, _ = self.upper_model.predict_f(x)
        return self._ci_scale(x, f_mean.numpy(), assimilate_factor)

    def get_middle(self, x):
        """Return the IQM estimate of y at the position(s) x."""
        f_mean, _ = self.iqm_model.predict_f(x)
        return f_mean.numpy()

    def get_lower(self, x, assimilate_factor: float = 1.0):
        """Return the lower CI estimate of y at the position(s) x."""
        f_mean, _ = self.lower_model.predict_f(x)
        return self._ci_scale(x, f_mean.numpy(), assimilate_factor)

    @staticmethod
    def get_model_name() -> str:
        """Return name of this model, for naming files and the like."""
        return "igpr_"
    
    def add_viz_info(self, viz_info) -> None:
        """Add a visualization to this model."""
        self._viz_infos.append(viz_info)

    def get_viz_infos(self):
        """Return visualization info(s) for data points used for training the model."""
        return self._viz_infos
    
    def build_df(self, x, y, y_axis_label: str):
        """Helper to construct a `DataFrame` given x points and y readings and a label for y.

        Labels for x are taken from the model.
        """
        assert x.shape[1] == len(self.dim_info)
        return DataFrame(np.concatenate([x, y], axis=1), columns=self.get_ls_dim_names() + [y_axis_label])
    
    def _ci_scale(self, x, y, assimilate_factor: float = 1.0):
        """Assimilate passed y values (assumed to come from `get_upper` or `get_lower`) towards the middle values."""
        if assimilate_factor == 1.0:
            return y

        y_middle = self.get_middle(x)
        return assimilate_factor * y + (1 - assimilate_factor) * y_middle
    
    def get_ls_dim_names(self) -> list[str]:
        """Get the list of hyperparameter landscape dimension names."""
        return self.hp_names
    
    def get_dim_info(self, name: str):
        """Return matching `DimInfo` to a passed name (can be y_info of any dim_info)."""
        return None

<>:126: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:126: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/3067408469.py:126: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if c is not "fold":


In [15]:
def plot_figure(
    fig,
    global_gs,
    fig_file_part: str | None,
    row_titles: list[str],
    height_ratios: list[float],
    add_legend: bool = True,
) -> None:
    """Show or save the figure after adding row titles."""
    # add titles:
    title_gs = GridSpecFromSubplotSpec(1 + len(row_titles), 1, global_gs[0], height_ratios=height_ratios)
    for i, row_title in enumerate(row_titles, start=1):
        ax = fig.add_subplot(title_gs[i, 0])
        ax.text(1.0, 0.5, row_title, ha="right", va="center", fontsize=TITLE_FSIZE)
        ax.axis("off")

    if add_legend:
        # get unique labels in whole figure:
        foo = [a.get_legend_handles_labels() for a in fig.axes]
        handles, labels = (sum(f, []) for f in _transpose(foo))
        by_label = dict(zip(labels, handles))
        fig.legend(by_label.values(), by_label.keys(), loc="center right", fontsize=LEGEND_FSIZE)

    if fig_file_part is None:
        plt.show()
    else:
        Path(fig_file_part).parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(f"{fig_file_part}.pdf", bbox_inches="tight")

In [16]:
def load_data(algorithm, env):
    source_file = f"../results_combined/sobol/{env}_{algorithm}.csv"
    return pd.read_csv(source_file)

In [17]:
def get_phase(data, step_count):
    phase_data = data[data["budget"] == step_count]
    best_config = phase_data.iloc[phase_data["performance"].idxmax()]
    best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)
    n_dims = len(best_config)
    phase_data["conf.ls.dims"] = n_dims
    #best_config["conf.l"]
    return phase_data, best_config

In [21]:
for algorithm in ["ppo", "dqn", "sac"]:
    for env in envs:
        if os.path.exists(f"../results_combined/sobol/{env}_{algorithm}.csv"):
            data = load_data(algorithm, env)
            default_configspace = algorithms[algorithm].get_hpo_search_space().get_hyperparameter_names()
            configspace = ConfigurationSpace()
            for c in default_configspace:
                if c in ["buffer_prio_sampling", "use_target_network", "alpha_auto", "normalize_advantage"]:
                    hp = Categorical(c, [True, False])
                    configspace.add_hyperparameter(hp)
                else:
                    key = [a for a in data.keys() if c in a]
                    if len(key) > 0:
                            key = key[0]
                            hp_min = data[key].min()
                            hp_max = data[key].max()
                            bounds = (min(0, hp_min-0.1*hp_min), hp_max+0.1*hp_max)
                            hp = Float(c, bounds=bounds)
                            configspace.add_hyperparameter(hp)
            Y_BOUNDS = (-200, 200)
            fitdata = []
            phases = [1e7]
            try:
                for phase_index, phase_steps in enumerate(phases):#enumerate([25e5, 5e6, 75e5, 1e7]):
                    phase_data, best_conf = get_phase(data, phase_steps)
                    model = TripleGPModel(phase_data, np.float64, "performance", Y_BOUNDS, None, list(best_conf.keys()), configspace)
                    model.fit()
                    _fitdata = model.estimate_iqm_fit()
                    _fitdata["phase_index"] = 0
                    fitdata.append(_fitdata)
                fitdata = pd.concat(fitdata)
                fitdatabase = Path("igpr_data") / f"{env}_{algorithm}"
                fn = Path(fitdatabase) / "iqm_fit_cv.csv"
                fn.parent.mkdir(parents=True, exist_ok=True)
                fitdata.to_csv(fn, index=False)
            except Exception as e:
                print(f"Error: {e}")


/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)


--------------------------------------------------
Estimate IQM surface fit
mean_squared_error 0.024100631445427432 0.01916572737953635
mean_absolute_error 0.09603315147842877 0.024603049844979224


/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)


--------------------------------------------------
Estimate IQM surface fit
mean_squared_error 0.010140194354252377 0.009758317728050067
mean_absolute_error 0.0496645400943065 0.017101379675455025


/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)


--------------------------------------------------
Estimate IQM surface fit
mean_squared_error 0.0006784709741296362 0.00013466217307856913
mean_absolute_error 0.018494022467655864 0.0014306625065327733


/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)


--------------------------------------------------
Estimate IQM surface fit
mean_squared_error 0.03573926680959626 0.006549050302826159
mean_absolute_error 0.1552380868865751 0.01575777281148899
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: attempt to get argmax of an empty sequence
Error: 0
Error: 0
Error: 0
Error: 0
Error: attempt to get argmax of an empty sequence
Error: attempt to g

/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)
/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)
/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_69434/2006587659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_config.drop(["budget", "performance", "seed", "run_id"], inplace=True)


--------------------------------------------------
Estimate IQM surface fit
mean_squared_error 0.024371135743858786 0.005167066333945617
mean_absolute_error 0.14248460347454645 0.018418452677439544


In [ ]:
fig = plt.figure(figsize=(100, 100))
global_gs = fig.add_gridspec(1, 1 + len(phases))
row_titles, height_ratios = visualize_nd(
                        model, fig, global_gs[0], 1, viz_group="maps", phase_index=phase_index
                    )
plot_figure(fig, global_gs, Path("igpr_data"), row_titles, height_ratios, add_legend=True)

/Users/theeimer/anaconda3/envs/arlbench/lib/python3.10/site-packages/autorl_landscape/visualize.py:480: FutureWarning: The provided callable <function mean at 0x106f06680> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pt = data.pivot_table(values=y_col_name, index=x0, columns=x1, aggfunc=np.mean)
/Users/theeimer/anaconda3/envs/arlbench/lib/python3.10/site-packages/autorl_landscape/visualize.py:481: FutureWarning: The provided callable <function mean at 0x106f06680> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pt_T = data.pivot_table(values=y_col_name, index=x1, columns=x0, aggfunc=np.mean)
/Users/theeimer/anaconda3/envs/arlbench/lib/python3.10/site-packages/autorl_landscape/visualize.py:480: FutureWarning: The provided callable